# End to End Chicken Disease Classification Project

## Stage 3 : Model Training

In [1]:
%pwd

'd:\\Documents\\Projects\\chiicken_disease_classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\Documents\\Projects\\chiicken_disease_classification'

### Update entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

### Update configuration manager in src config

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILEPATH,
                 params_filepath=PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir,
            "Chicken-fecal-images"
        )

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path), updated_model_path=Path(prepare_base_model.updated_model_path),
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_is_augmentation=params.AUGMENTATION
        )

        return training_config

### Update components

In [8]:
import tensorflow as tf


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_model_path
        )

    def prepare_data_for_training(self):
        train_ds = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="training",
            seed=42,
            batch_size=self.config.params_batch_size,
            image_size=(
                self.config.params_image_size[0], self.config.params_image_size[1])
        )

        val_ds = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="validation",
            seed=42,
            batch_size=self.config.params_batch_size,
            image_size=(self.config.params_image_size[0], self.config.params_image_size[1]))

        self.train_ds = self.configure_for_performance(train_ds)
        self.val_ds = self.configure_for_performance(val_ds)

    def train(self):
        self.model.fit(
            self.train_ds,
            epochs=self.config.params_epochs,
            # batch_size=self.config.params_batch_size,
            # steps_for_epoch=len(self.train_ds),
            validation_data=self.val_ds,
            # validation_steps=len(self.val_ds)
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    def configure_for_performance(self, ds):
        ds = ds.cache()
        ds = ds.shuffle(buffer_size=1000)
        # ds = ds.batch(self.config.params_batch_size)
        ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
        return ds

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

### Update pipeline

In [9]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.prepare_data_for_training()
    training.train()
except Exception as e:
    raise e

[2023-10-19 11:39:55,987:  INFO: common: yaml file: config\config.yaml loaded]
[2023-10-19 11:39:55,995:  INFO: common: yaml file: params.yaml loaded]
[2023-10-19 11:39:56,000:  INFO: common: Created directory at artifacts]
Found 390 files belonging to 2 classes.
Using 312 files for training.
Found 390 files belonging to 2 classes.
Using 78 files for validation.


ValueError: in user code:

    File "d:\Documents\Projects\chiicken_disease_classification\venv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "d:\Documents\Projects\chiicken_disease_classification\venv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Documents\Projects\chiicken_disease_classification\venv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "d:\Documents\Projects\chiicken_disease_classification\venv\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "d:\Documents\Projects\chiicken_disease_classification\venv\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    Fi